### Data Analytics Challenge - Mercado Livre - ETL

#### Criar um modelo para clusterização dos vendedores do mercado livre para o time de estratégia criar ações específicas a cada tipo de vendedor.

Objetivo: criar um cluster onde seja possível direcionar estratégias específicas a cada grupo de sellers dentro do ML Argentina

A extração da informação deve ser realizada através de API do ML:

a) API de Categorias;
b) API de Itens;
c) API de Vendedores;
d) API de Reputação

Informações necessárias a serem extraídas:

a) Categorias;
b) Dentro dessas categorias, trazer a lista de itens
c) Quantidade de itens distintos vendidos por cada vendedor;
d) Trazer a média de preço dos itens e estimar a receita produzida por vendedor;
e) Enriquecer a base com a reputação dos vendedores e total de transacoes



In [1]:
import requests
import json

# Definir o refresh token
refresh_token = "TG-67ba3fd37af6fb00016f58ae-200275757"

# URL da API para renovação do token
url = "https://api.mercadolibre.com/oauth/token"

# Parâmetros para a requisição
payload = f'grant_type=refresh_token&client_id=6468562705974725&client_secret=VpxkqqgvdNb5HNZSsMG8fJvB80HKKitT&refresh_token={refresh_token}'
headers = {
  'accept': 'application/json',
  'content-type': 'application/x-www-form-urlencoded'
}

# Fazer a requisição para renovar o token
response = requests.post(url, headers=headers, data=payload)

# Verificar se a requisição foi bem-sucedida
if response.status_code == 200:
    response_data = response.json()  # Converte a resposta JSON para um dicionário
    
    # Extrair o novo access token
    access_token = response_data.get("access_token", "Não encontrado")
    
    print("Novo Access Token:", access_token)  # Exibir o access token
else:
    print("Erro ao obter o token:", response.status_code, response.text)


Novo Access Token: APP_USR-6468562705974725-022616-ed6dbe03ff7dd779ea9e0ba659962357-200275757


In [3]:
# Importando as bibliotecas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import time


### 1) Trazer as categorias da Argentina


In [4]:
# URL para obter todas as categorias da Argentina
CATEGORIES_URL = "https://api.mercadolibre.com/sites/MLA/categories"

# Fazer a requisição para obter a lista de categorias
response = requests.get(CATEGORIES_URL)
categorias = response.json()

# Converter para DataFrame
df_categorias = pd.DataFrame(categorias)
df_categorias

,id,name
0,MLA5725,Accesorios para Vehículos
1,MLA1512,Agro
2,MLA1403,Alimentos y Bebidas
3,MLA1071,Animales y Mascotas
4,MLA1367,Antigüedades y Colecciones
5,MLA1368,"Arte, Librería y Mercería"
6,MLA1743,"Autos, Motos y Otros"
7,MLA1384,Bebés
8,MLA1246,Belleza y Cuidado Personal
9,MLA1039,Cámaras y Accesorios


### 2) Lista dos vendedores e produtos disponiveis na Argentina

In [5]:
# Definição do Access Token para autenticação
HEADERS = {"Authorization": f"Bearer {access_token}"}

# URL base para buscar categorias
CATEGORIES_URL = "https://api.mercadolibre.com/sites/MLA/categories"

# Taxa fixa de conversão ARS → USD (atualizada conforme necessário)
ARS_TO_USD = 0.0012  # Caso precise, atualize este valor manualmente

# Fazer a requisição para obter a lista de categorias
response = requests.get(CATEGORIES_URL)
categorias = response.json()

# Lista para armazenar os resultados
vendedores_lista = []

# Iterar sobre as categorias para obter vendedores e produtos
for categoria in categorias:
    category_id = categoria["id"]
    category_name = categoria["name"]
    
    # Paginação para coletar o máximo possível de vendedores por categoria
    for offset in range(0, 1000, 50):  # Ajustável conforme necessidade
        search_url = f"https://api.mercadolibre.com/sites/MLA/search?category={category_id}&limit=50&offset={offset}"
        
        try:
            response = requests.get(search_url)
            response.raise_for_status()  # Lança erro se houver problema na API
            data = response.json()
        except requests.exceptions.RequestException as e:
            print(f"⚠️ Erro ao acessar API para categoria {category_id}: {e}")
            continue
        
        # Iterar sobre os produtos retornados e extrair informações do vendedor e produtos
        for produto in data.get("results", []):
            vendedor = produto.get("seller", {})
            item_id = produto.get("id", "N/A")

            # Obter a quantidade de estoque diretamente de `available_quantity`
            quantidade_estoque = produto.get("available_quantity", 0)

            # Garantir que o preço seja um número e não None
            preco_ars = produto.get("price") if produto.get("price") is not None else 0

            # Calcular valor total de estoque
            valor_total_estoque_ars = preco_ars * quantidade_estoque

            # Conversão para USD com taxa fixa
            preco_usd = preco_ars * ARS_TO_USD
            valor_total_estoque_usd = valor_total_estoque_ars * ARS_TO_USD
            
            # Adicionar ao dataframe
            vendedores_lista.append({
                "categoria": category_id,
                "nome_categoria": category_name,
                "id_vendedor": vendedor.get("id", "N/A"),
                "nome_vendedor": vendedor.get("nickname", "Desconhecido"),
                "id_produto": item_id,
                "nome_produto": produto.get("title", "Desconhecido"),
                "preco_ars": preco_ars,
                "preco_usd": round(preco_usd, 2),
                "quantidade_estoque": quantidade_estoque,
                "valor_total_estoque_ars": valor_total_estoque_ars,
                "valor_total_estoque_usd": round(valor_total_estoque_usd, 2)
            })
        
        # Parar se não houver mais produtos para carregar
        if len(data.get("results", [])) < 50:
            break

# Converter para DataFrame
df_vendedores = pd.DataFrame(vendedores_lista)

In [8]:
df_vendedores.to_excel(r'C:\Users\act\Downloads\df_resultado_clusters_.xlsx', index = False)

### 3) Reputação dos vendedores e quantidade de transações

In [9]:
# Definição do Access Token para autenticação
# URL base para buscar reputação dos vendedores
REPUTATION_URL = "https://api.mercadolibre.com/users/{seller_id}"

# Lista para armazenar os resultados
vendedores_lista = []

# Iterar sobre os vendedores extraídos anteriormente
for seller_id in df_vendedores["id_vendedor"].unique():
    
    # Requisição para obter a reputação do vendedor
    rep_response = requests.get(REPUTATION_URL.format(seller_id=seller_id), headers={"Authorization": f"Bearer {access_token}"})
    rep_data = rep_response.json()
    
    # Obter informações relevantes
    reputacao = rep_data.get("seller_reputation", {}).get("level_id", "N/A")
    power_seller_status = rep_data.get("seller_reputation", {}).get("power_seller_status", "N/A")
    total_transacoes = rep_data.get("seller_reputation", {}).get("transactions", {}).get("total", 0)    
    vendedores_lista.append({
        "id_vendedor": seller_id,
        "nome_vendedor": df_vendedores.loc[df_vendedores["id_vendedor"] == seller_id, "nome_vendedor"].values[0],
        "reputacao": reputacao,
        "status_vendedor": power_seller_status,
        "total_transacoes": total_transacoes
    })

# Converter para DataFrame
df_vendedores_reputacao = pd.DataFrame(vendedores_lista)

# Exibir a lista de vendedores com reputação

### 4) Agrupamento da tabela dos vendedores, trazendo quantidade distinta de produtos listados e média de preço



In [10]:
# Taxa de conversão ARS para USD
ARS_TO_USD = 0.0012  

# Agrupar df_vendedores por categoria, nome_categoria, id_vendedor, nome_vendedor
df_vendedores_agrupado = df_vendedores.groupby([ "id_vendedor", "nome_vendedor"]).agg(
    categorias_atendidas=("categoria","nunique"),
    qtde_distinta_produtos=("id_produto", "nunique"),   # Contar produtos distintos por vendedor
    media_preco=("preco_usd", "mean"),                      # Média dos preços dos produtos
    qtde_disponivel_estoque=("quantidade_estoque", "sum"), #Total disponível em estoque
    total_valor_estoque_usd=("valor_total_estoque_usd","sum") #Total monetário em estoque
).reset_index()

# Adicionar conversão para USD se os preços forem em ARS

df_vendedores_agrupado

,id_vendedor,nome_vendedor,categorias_atendidas,qtde_distinta_produtos,media_preco,qtde_disponivel_estoque,total_valor_estoque_usd
0,100471,TECNO-OUTLET,1,1,4.200,1,4.20
1,101217,WSBTECHNO,1,1,5.970,1,5.97
2,104410,MONGETUNSA,1,1,7.680,1,7.68
3,110162,COMPUFANSTORE,1,1,114.800,500,57399.37
4,111286,WACKY_,1,5,22.040,1551,32036.93
...,...,...,...,...,...,...,...
4167,2228196784,SHEFA_INTERNACIONAL,1,2,4.565,51,311.71
4168,2230896619,CS20250125180930,1,1,392.400,1,392.40
4169,2236152086,SHAJUAR_ENDIN,2,2,12.640,2,25.28
4170,2243047476,NEWADNTECH,1,1,59.880,1,59.88


### 5) Consolidando as informações com a reputação dos vendedores

In [11]:
# Fazer o merge usando "id_vendedor" como chave
df_final = df_vendedores_agrupado.merge(df_vendedores_reputacao[['id_vendedor','reputacao','status_vendedor','total_transacoes']], on="id_vendedor", how="left")
df_final["vendas_estimativa"] = df_final['total_transacoes'] * df_final['media_preco']

In [12]:
df_final.to_csv(r'C:\Users\act\Downloads\analise_vendedores.csv', sep=';', index = False, decimal =',', encoding = 'cp1252')